Imports

In [103]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


Get userdata

In [104]:
df_userdata = pd.read_csv('../assets/userdata_min_40_v4.csv')
df_userdata = df_userdata.drop(df_userdata.columns[1:11], axis=1)
df_userdata

,user_id,kokosmelk,water,chilivlokken,gemalen kaneel,sjalot,knoflook,rode peper,zout,vastkokende aardappel,...,tomatenblokjes,zoete puntpaprika,(olijf)olie,kruimige aardappelen,magere gerookte spekreepjes,takje rozemarijn,boter,aardappelen,kitchen,liked_recipes
0,1,0,0,1,1,1,1,0,1,0,...,0,0,0,0,0,1,0,0,frans,NaN
1,2,0,0,0,0,1,1,0,0,1,...,0,0,0,0,0,0,0,0,frans,NaN
2,3,0,1,0,0,1,1,0,1,0,...,0,0,0,0,1,0,0,0,frans,NaN
3,4,0,0,0,0,0,1,0,1,0,...,0,0,0,1,0,0,1,0,frans,NaN
4,5,0,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,frans,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599,600,0,0,0,1,0,1,0,0,0,...,1,0,1,0,0,1,0,0,mediterraan,NaN
600,601,0,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,geen,"[541, 45, 958, 3356, 4008, 1394, 4138, 4190, 6..."
601,602,0,1,0,0,0,1,0,1,0,...,1,0,0,0,0,0,0,0,geen,"[48, 4189, 792, 2082, 4512, 4556, 379, 721, 84..."
602,603,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,geen,"[2919, 1376, 1697, 4323, 2502, 4011, 3264, 356..."


Get all ingredients

In [105]:
df_recipes_min_40 = pd.read_csv("../assets/recipes_min_40.csv")

filtered_recipes = df_recipes_min_40
first_ingredient_index = filtered_recipes.columns.get_loc("tag_end") + 1

all_ingredients = df_recipes_min_40.columns.values[first_ingredient_index:].tolist()
print(all_ingredients)

['kokosmelk', 'water', 'chilivlokken', 'gemalen kaneel', 'sjalot', 'knoflook', 'rode peper', 'zout', 'vastkokende aardappel', 'zoete aardappel', 'vissaus', 'middelgrote ui', 'ongezouten roomboter', 'droge witte wijn', 'verse slagroom', 'limoen', 'arachideolie', 'ketjap manis', 'bloemkool', 'rundergehakt', 'tarwebloem', 'gemalen nootmuskaat', 'middelgroot ei', 'paprikamix', 'trostomaat', 'mozzarella', 'salade-ui', 'spaghetti', 'milde olijfolie', 'balsamicoazijn', 'komkommer', 'cherrytomaten', 'broccoli', 'champignons', 'verse peterselie', 'verse koriander', 'winterpeen', 'babyromainesla', 'citroen', 'verse spinazie', 'gesneden uien', 'romaatjes', 'halfvolle melk', 'groentebouillontablet', 'kokend water', 'taugé', 'rode ui', 'kipdijfilet', 'zonnebloemolie', 'couscous', 'eetrijpe avocado', 'snoepgroente tomatenmix', 'prei', 'verse tijm', 'verse platte peterselie', 'bloem', 'kastanjechampignon', 'ketchup', 'kruimige aardappel', 'rucola', 'knolselderij', 'verse bieslook', 'dijonmosterd', 'c

Convert userdata dataframe to strings

In [106]:
df_converted_userdata = pd.DataFrame(columns=['user_id', 'ingredients'])

for index, row in df_userdata.iterrows():
    user_ingredients = ""

    for ingredient in all_ingredients:
        if row[ingredient] == 1:
            user_ingredients += " " + ingredient

    user_row = {'user_id': row['user_id'], 'ingredients': user_ingredients, 'liked_recipes': row['liked_recipes']}
    df_converted_userdata = pd.concat([df_converted_userdata, pd.DataFrame.from_records([user_row])], ignore_index=True)

df_converted_userdata

,user_id,ingredients,liked_recipes
0,1,chilivlokken gemalen kaneel sjalot knoflook z...,NaN
1,2,sjalot knoflook vastkokende aardappel ongezou...,NaN
2,3,water sjalot knoflook zout ongezouten roombot...,NaN
3,4,knoflook zout ongezouten roomboter droge witt...,NaN
4,5,knoflook vastkokende aardappel arachideolie m...,NaN
...,...,...,...
599,600,gemalen kaneel knoflook arachideolie bloemkoo...,NaN
600,601,sjalot knoflook ongezouten roomboter verse sl...,"[541, 45, 958, 3356, 4008, 1394, 4138, 4190, 6..."
601,602,water knoflook zout ongezouten roomboter arac...,"[48, 4189, 792, 2082, 4512, 4556, 379, 721, 84..."
602,603,gemalen kaneel knoflook zoete aardappel droge...,"[2919, 1376, 1697, 4323, 2502, 4011, 3264, 356..."


Add user row to recipes (to compare)

In [107]:
import ast

df_converted_recipes = pd.DataFrame(columns=['index', 'name'])

recipe_row = {'index': 0, 'name': 'user_row', 'ingredients': df_converted_userdata.iloc[-1].ingredients}
df_converted_recipes = pd.concat([df_converted_recipes, pd.DataFrame.from_records([recipe_row])], ignore_index=True)

df_converted_recipes

,index,name,ingredients
0,0,user_row,chilivlokken knoflook ongezouten roomboter mi...


Convert recipe dataframe to strings

In [108]:
liked_recipes_indexes = ast.literal_eval(df_converted_userdata.iloc[-1].liked_recipes)

for index, row in df_recipes_min_40.iterrows():

    recipe_ingredients = ""

    for ingredient in all_ingredients:
        if row[ingredient] == 1:
            recipe_ingredients += " " + ingredient

    recipe_row = {'index': row['index'] + 1, 'name': row['name'], 'ingredients': recipe_ingredients}

    if row['index'] not in liked_recipes_indexes:
        df_converted_recipes = pd.concat([df_converted_recipes, pd.DataFrame.from_records([recipe_row])], ignore_index=True)

df_converted_recipes

,index,name,ingredients
0,0,user_row,chilivlokken knoflook ongezouten roomboter mi...
1,1,Massaman curry,kokosmelk water chilivlokken gemalen kaneel s...
2,2,Tartiflette,knoflook vastkokende aardappel middelgrote ui...
3,3,Ajam pangang,water sjalot knoflook rode peper zout ongezou...
4,4,Bloemkoolschotel,knoflook ongezouten roomboter bloemkool runde...
...,...,...,...
4580,4590,Thaise kipcurry met aubergine,kokosmelk zonnebloemolie aubergine basmatirijst
4581,4591,Romige spaghetti met zalmreepjes,spaghetti cherrytomaten courgette traditionel...
4582,4592,Tijmballetjes met puree,halfvolle melk paneermeel boter aardappelen
4583,4593,Romige mosselen met dille,droge witte wijn grove mosterd bleekselderij ...


Compare user ingredients to recipes

In [111]:
combined_features = df_converted_recipes['ingredients']

vectorizer = TfidfVectorizer()
feature_vectors = vectorizer.fit_transform(combined_features)

similarity = cosine_similarity(feature_vectors)

similarity_score = list(enumerate(similarity[0]))
sorted_similar_recipes = sorted(similarity_score, key=lambda x: x[1], reverse=True)

print('Recipes suggested for you : \n')

i = 1
for recipe in sorted_similar_recipes[1:]:
    index = recipe[0]
    name_from_index = df_converted_recipes[df_converted_recipes.index == index]['name'].values[0]
    if i <= 10:
        print(str(i) + '.', name_from_index)
        i += 1


Recipes suggested for you : 

1. Lentequiches met pecorino en ham
2. Snelle Spaanse tomatenstoof met gehaktballetjes
3. Ovenschotel met gehakt en spitskool
4. Flammkuchen met gruyère en groene asperges
5. Cevapcici (worstjes uit de balkan)
6. Gnocchi met mosterd-roomsaus
7. Pasta met kipballetjes en tomaat uit de oven
8. Gnocchi carbonara
9. Hollandse makreelkoekjes met maiskolven 
10. Gebakken witlof met romige kruidenpuree
